### Histopathologic Cancer Detection
Identify metastatic tissue in histopathologic scans of lymph node sections



In [ ]:
# Libraries
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [ ]:
# Parameters for model.

# Hyper parameters
NUM_EPOCHS = 8
NUM_CLASSES = 2
BATCH_SIZE = 128
LEARNING_RATE = 0.002

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
labels = pd.read_csv('/content/input/train_labels.csv')
sub = pd.read_csv('/content/input/sample_submission.csv')
train_path = '/content/input/train/'
test_path = '/content/input/test/'

In [ ]:
# splitting data into train and Val
train, val = train_test_split(labels, stratify=labels.label, test_size=0.1)
len(train), len(val)

### Simple Custom data generator

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df_data, data_dir = './', transform=None):
        super().__init__()
        self.df = df_data.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_name,label = self.df[index]
        img_path = os.path.join(self.data_dir, img_name)
        image = cv2.imread(img_path)
        if self.transform is not None:
            image = self.transform(image)
        return image, label

In [ ]:
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                       std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Pad(64, padding_mode='reflect'),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                       std=[0.5, 0.5, 0.5])])


dataset_train = MyDataset(df_data=train, data_dir='', transform=None)
dataset_valid = MyDataset(df_data=val, data_dir=train_path, transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=BATCH_SIZE, 
                          shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=BATCH_SIZE//2, 
                          shuffle=False, num_workers=0)


### Model

In [ ]:
class SimpleCNN(nn.Module):

  def __init__(self):
    # ancestor constructor call
    super(SimpleCNN, self).__init__()

    # Convolution2d layer
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, 
                           kernel_size=3, padding=2)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64,
                           kernel_size=3, padding=2)
    self.conv3 = nn.Conv2d(in_channels=64, out_channels=128,
                           kernel_size=3, padding=2)
    self.conv4 = nn.Conv2d(in_channels=128, out_channels=256,
                           kernel_size=3, padding=2)
    self.conv5 = nn.Conv2d(in_channels=256, out_channels=512,
                           kernel_size=3, padding=2)
    
    # batch normalization
    self.bn1 = nn.BatchNorm2d(32)
    self.bn2 = nn.BatchNorm2d(64)
    self.bn3 = nn.BatchNorm2d(128)
    self.bn4 = nn.BatchNorm2d(256)
    self.bn5 = nn.BatchNorm2d(512)

    # Max pooling
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

    # Avg pooling
    self.avg = nn.AvgPool2d(8)

    # fully connected layer
    self.fc = nn.Linear(512*1*1, 2)

    # fwd layer
    def forward(self, x):

      # Conv layer -> Batch Norm -> Activation -> Pooling
      x = self.pool(F.leaky_relu(self.bn1(self.conv1(x))))
      x = self.pool(F.leaky_relu(self.bn2(self.conv2(x))))
      x = self.pool(F.leaky_relu(self.bn3(self.conv3(x))))
      x = self.pool(F.leaky_relu(self.bn4(self.conv4(x))))
      x = self.pool(F.leaky_relu(self.bn5(self.conv5(x))))
      x = self.avg(x)
      print(x.shape)

      # size (batch_size, C, H, W)
      x = x.view(-1, 512*1*1)
      x = self.fc(x)
      return x

In [ ]:
model = SimpleCNN().to(device)

In [ ]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=LEARNING_RATE)

In [ ]:
# Train the model
total_step = len(loader_train)
for epoch in range(NUM_EPOCHS):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

### Accuracy Check

In [ ]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
          
    print('Test Accuracy of the model on the 22003 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'trained_model.ckpt')

### CSV Submission

In [ ]:
dataset_valid = MyDataset(df_data=sub, data_dir=test_path, transform=trans_valid)
loader_test = DataLoader(dataset = dataset_valid, batch_size=32, shuffle=False, num_workers=0)

In [ ]:
model.eval()

preds = []
for batch_i, (data, target) in enumerate(loader_test):
    data, target = data.cuda(), target.cuda()
    output = model(data)

    pr = output[:,1].detach().cpu().numpy()
    for i in pr:
        preds.append(i)
sub.shape, len(preds)
sub['label'] = preds
sub.to_csv('submission_file.csv', index=False)